In [1]:
import pymssql
conn = pymssql.connect(server="47.98.41.112",user="wj_read",password="d6dfyl5h",database="fyhd_db")

In [2]:
import pandas as pd

In [3]:
# # 选择时间
# beginTime="'2019-04-01'"
# endTime="'2019-04-017'"
# import pymssql
# conn = pymssql.connect(server="47.98.41.112",user="wj_read",password="d6dfyl5h",database="fyhd_db")
# import pandas as pd

## 运营

In [4]:
#1订单导入及时率汇总
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int 
select @beginTime='2019-05-1', @endTime='2019-05-14'
select t2.managementName as '归属',t.distributionTime as '订单配送开始日期',sum(t.totalCount) as '订单总数',sum(t.count2) '订单导入时间小于配送开始时间订单数' from 
(
select  
tb_company.company_id,
tb_company.company_name,
convert(varchar(10),distributionTime,120) as distributionTime,
count(orderId) as totalCount , 
sum(case when createTime < distributionTime then 1 else 0 end) as count2
from CodOrder 
LEFT JOIN tb_company on tb_company.company_Id = codorder.companyId
where companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') and orderTransportationType = 1 
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
and orderTransportationType=1 --只取同城配送，专线运输
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)
and distributionTime >= @beginTime and distributionTime < @endTime  and deleteflag = 0 and orderState <> 4 and CodOrder.clientId != 20071
and codorder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '订单导入及时率')-- 无单配车剔除
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')

group by convert(varchar(10),distributionTime,120),tb_company.company_name,tb_company.company_id 
) t 
LEFT JOIN OnLineCompanyManagement t1 on t.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.distributionTime,t.company_id
order by t2.managementName,convert(varchar(10),t.distributionTime,120)
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总订单导入及时率.xlsx')

In [5]:
#1订单导入及时率明细
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int 
select @beginTime='2019-05-1', @endTime='2019-05-14'
select  
t2.managementName as '归属',
tb_company.company_name as '公司名称',
convert(varchar(10),distributionTime,120) as '订单配送开始日期',
CodOrder.orderNo  as '订单号',
CodClient.clientName as '客户名称',
CodClient.clientId as '客户id',
(case  CodOrder.orderTransportationType when 1 then '同城配送' when 2 then '专线运输' when 3 then '无需配送' end ) as '配送类型',
CodOrder.createTime as '订单创建时间',
Codorder.distributionTime as '订单配送开始时间',
CodOrder.distributionTimeEnd as '订单配送结束时间',
case when CodOrder.createTime < CodOrder.distributionTime then '正常' else '订单导入时间小于配送开始时间' END as '是否异常'
from CodOrder 
LEFT JOIN tb_company on tb_company.company_Id = codorder.companyId
LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId

LEFT JOIN OnLineCompanyManagement t1 on tb_company.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') and CodOrder.orderTransportationType = 1
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
and codorder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '订单导入及时率')-- 无单配车剔除
and orderTransportationType =1  --同城配送
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)

and distributionTime >= @beginTime and distributionTime < @endTime  and CodOrder.deleteflag = 0 and CodOrder.orderState <> 4 and  CodOrder.clientId != 20071
order by codorder.createTime 
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细订单导入及时率.xlsx')

In [6]:
#2调度系统派单率汇总
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
t2.managementName as '归属',
tb_company.company_name,
convert(varchar(10),distributionTime,120) as '日期',
count(orderId) as '应派车订单总数'
,sum(case when currentSendcarId > 0 then 1 else 0 end) as '调度系统完成派车的订单总数'
from CodOrder 
LEFT JOIN tb_company on tb_company.company_id = CodOrder.companyId

LEFT JOIN OnLineCompanyManagement t1 on tb_company.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
and distributionTime >= @beginTime and  distributionTime < @endTime
and CodOrder.deleteflag = 0 and orderState <> 4 and orderTransportationType = 1 and CodOrder.clientId != 20071
and orderTransportationType =1 --同城配送
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)
 and (case when CodOrder.createType !='WebService' and CodOrder.orderCause!='配送' and CodOrder.clientId=19483 then 0 else 1 end=1)
 and (case when CodOrder.createType ='WebService' and CodOrder.currentDriverId is null  then 0 else 1 end=1)
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
group by convert(varchar(10),distributionTime,120),tb_company.company_name,t2.managementName
order by tb_company.company_name,convert(varchar(10),distributionTime,120)
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总调度系统派单率.xlsx')

In [7]:
#调度系统派单率明细
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
t2.managementName as '归属',
tb_company.company_name as '公司名称',
convert(varchar(10),distributionTime,120) as '日期',
CodClient.clientName as '客户名称',
CodSendcar.sendcarNo as '派车单号',
CodOrder.orderNo as '订单号',
CodSendcar.createTime as '派车单创建时间',
CodOrder.createTime as '订单创建时间',
Codorder.distributionTime as '订单配送开始时间',
CodOrder.distributionTimeEnd as '订单配送结束时间',
CodOrder.pickUpTime as '提货时间',
CodOrder.warehouseArrivalTime    as '到仓时间',
CodOrder.createType as '创建途径',
CodOrder.orderCause as '订单类型',
CodOrder.businessType as '无单派车',
( case CodOrder.orderState when 0 then '新订单' when 1 then '待提货' when 2 then '配送中'
when 3 then '已完成' when 4 then '已作废' when 5 then '已返货' end) as '订单状态'  ,
(case  when  CodOrder.firstSendcarTime is not null then '是' else '否' end ) as '是否派车',
CodOrder.endLinkman as '收货门店',
(case CodSendcar.exeTimeType when 0 then '即时派车' when 1 then '计划派车' when 2 then '无单派车' end) as '派车类型' ,
case when currentSendcarId > 0 then '正常' else '未派车' end as '是否派车'
from CodOrder
LEFT JOIN CodSendcar on CodSendcar.sendcarId = CodOrder.currentsendcarId
LEFT JOIN tb_company on tb_company.company_id = CodOrder.companyId
LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId

LEFT JOIN OnLineCompanyManagement t1 on tb_company.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
and distributionTime >= @beginTime and  distributionTime < @endTime
and CodOrder.deleteflag = 0 and orderState <> 4 and orderTransportationType = 1 and CodOrder.clientId != 20071
and orderTransportationType=1 --同城配送
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)
and (case when CodOrder.createType !='WebService' and CodOrder.orderCause!='配送' and CodOrder.clientId=19483 then 0 else 1 end=1)
and (case when CodOrder.createType ='WebService' and CodOrder.currentDriverId is null  then 0 else 1 end=1)
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细调度系统派单率.xlsx')

In [8]:
#调度系统派单及时率汇总
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
t2.managementName as '归属',
tb_company.company_name,
convert(varchar(10),distributionTime,120) as '日期',
count(orderId) as '应派车订单总数'
,sum(case when ((currentSendcarId > 0 and CodOrder.distributionTime >= CodOrder.firstSendcarTime) or CodOrder.firstSendcarTime is null) then 1 else 0 end) as '调度系统及时派车的订单总数'
from CodOrder 
LEFT JOIN tb_company on tb_company.company_id = CodOrder.companyId
LEFT JOIN OnLineCompanyManagement t1 on tb_company.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
and distributionTime >= @beginTime and  distributionTime < @endTime
and CodOrder.createTime <= CodOrder.distributionTime
and CodOrder.deleteflag = 0 
and CodOrder.sendcarNos is not null
and orderState <> 4 
and orderTransportationType = 1 --同城配送
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)
and (case when CodOrder.createType !='WebService' and CodOrder.orderCause!='配送' and CodOrder.clientId=19483 then 0 else 1 end=1)
and CodOrder.clientId != 20071
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
group by convert(varchar(10),distributionTime,120),tb_company.company_name,t2.managementName
order by tb_company.company_name,convert(varchar(10),distributionTime,120)


"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总调度系统派单及时率.xlsx')

In [9]:
#调度系统派单及时率明细
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
t2.managementName as '归属',
tb_company.company_name as '公司名称',
convert(varchar(10),distributionTime,120) as '日期',
CodClient.clientName as '客户名称',
CodSendcar.sendcarNo as '派车单号',
CodOrder.orderNo as '订单号',
CodSendcar.createTime as '派车单创建时间',
CodSendcar.sendcarTime as '派车时间',
CodOrder.firstSendcarTime as '首次派车时间',
CodOrder.createTime as '订单创建时间',
Codorder.distributionTime as '订单配送开始时间',
CodOrder.distributionTimeEnd as '订单配送结束时间',
case when (currentSendcarId > 0 and  CodOrder.distributionTime >= CodOrder.firstSendcarTime) then '正常' when CodOrder.firstSendCarTime is null then '正常' else '非及时' end as '是否及时',
CodOrder.pickUpTime    as '提货时间',
CodOrder.warehouseArrivalTime    as '到仓时间'
from CodOrder
LEFT JOIN CodSendcar on CodSendcar.sendcarId = CodOrder.currentsendcarId
LEFT JOIN tb_company on tb_company.company_id = CodOrder.companyId
LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId
LEFT JOIN OnLineCompanyManagement t1 on tb_company.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
and distributionTime >= @beginTime and  distributionTime < @endTime
and CodOrder.createTime <= CodOrder.distributionTime
and CodOrder.deleteflag = 0 
and CodOrder.sendcarNos is not null
and orderState <> 4 
and orderTransportationType = 1  
and CodOrder.auditFlag<>0   -- 订单提交状态不等于未提交
and (CodOrder.businessType != '无单派车' or CodOrder.businessType is null)
and (case when CodOrder.createType !='WebService' and CodOrder.orderCause!='配送' and CodOrder.clientId=19483 then 0 else 1 end=1)
and CodOrder.clientId != 20071
AND CodOrder.clientId not  in (
SELECT codclients.clientId FROM CodClient  codclients
LEFT JOIN tb_company ON tb_company.company_id = CodOrder.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND codclients.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND codclients.deleteFlag = 0 AND codclients.clientState = 0
)
and CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
order by CodOrder.distributionTime 
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细调度系统派单及时率.xlsx')

In [10]:
#APP使用率汇总
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int 
select @beginTime='2019-05-1 00:00:00', @endTime='2019-05-13 23:59:59'
SELECT t2.managementName as '归属',t.da '日期', sum(t.orderNum) as '已指派司机的订单总数',sum(t.orderSum) as '司机APP完成签收订单总数' FROM
(
select 
tb_company.COMPANY_ID  ,
tb_company.company_name ,--as '公司名称',
convert(varchar(10),CodOrder.distributionTimeEnd,120) as da --as '日期' 
,count(Codorder.orderId) as orderNum --as '已指派司机的订单总数'
,sum(case when currentDriverId = signUserId then 1 else 0 end) orderSum --as '司机APP完成签收订单总数'
from CodOrder
LEFT JOIN tb_company on tb_company.company_id = codorder.companyId
LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId
where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') and  CodOrder.orderTransportationType = 1
AND  CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')

AND COdOrder.orderId not in (SELECT orderId from OnLineOrderData where indicatorsName='APP使用率')

AND CodClient.clientId not  in (
SELECT CodClient.clientId FROM CodClient 
LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
)
--and CodOrder.distributionTime >= @beginTime and CodOrder.distributionTime < @endTime  
and CodOrder.currentdriverId > 0
--and CodOrder.clientId not in (17953,18262,19083,19048,19327,19022,19398,19483,17992,18070,19843)
and CodOrder.deleteflag = 0   and CodOrder.orderState <> 4  
--and (distributionTimeEnd < GETDATE())   --配送截止时间小于当前时间,过滤掉跨天还在配送中的订单
--and (CONVERT(varchar(10),distributionTimeEnd,120) = @beginTime )
-- and (CONVERT(varchar(10),distributionTimeEnd,120) >= @beginTime )   and (CONVERT(varchar(10),distributionTimeEnd,120) < @endTime )  
and distributionTimeEnd >= @beginTime    and distributionTimeEnd < @endTime 
and CodOrder.distributionTIme > CodOrder.createTime
group by convert(varchar(10),CodOrder.distributionTimeEnd,120),tb_company.company_name,tb_company.COMPANY_ID
-- order by tb_company.company_name,convert(varchar(10),CodOrder.distributionTimeEnd,120);
)t
  
LEFT JOIN OnLineCompanyManagement t1 on t.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.da
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总APP使用率.xlsx')

In [11]:
#APP使用率明细
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
declare @companyId int 
select @beginTime='2019-05-1 00:00:00', @endTime='2019-05-13 23:59:59'
select 
tb_company.company_name as '公司名称',
CodClient.clientName as '客户名称',
CONVERT(varchar(10),CodOrder.distributionTimeEnd,120) as  '日期',
CodOrder.createTime as '订单创建时间',
CodSendcar.sendcarNo as '派车单号',
Codorder.orderNo as '订单编号',
Codorder.distributionTime as '订单配送开始时间',
CodOrder.distributionTimeEnd as '订单配送结束时间',
t1.real_name   as '司机',
t2.real_name  as '签收操作人',
CodOrder.signTime  as '实际签收时间',
tb_company.company_id,
case
when (signTime is not null and signTime <= distributionTimeEnd and currentDriverId = signUserId) then '正常-完成'     --配送截止时间小于当前时间,过滤掉跨天还在配送中的订单
when (signTime is null and distributionTimeEnd > GETDATE()) then '正常-配送中'     

when (signTime is not null and signTime > distributionTimeEnd and currentDriverId = signUserId) then '异常-配送迟到'
when (signTime is not null and currentDriverId <> signUserId) then '异常-签收非本人'
when (signTime is null) then '异常-未签收'
else '未知异常'  end  as signState ,--as '签收状态'
tt2.managementName '归属'
from CodOrder
LEFT JOIN tb_company on tb_company.company_id = codorder.companyId
LEFT JOIN OnLineCompanyManagement tt1 on tb_company.company_id = tt1.companyId
LEFT JOIN OnLineManagement tt2 on tt1.managementId = tt2.managementId
LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId
LEFT JOIN tb_user t1 on t1.user_id = codorder.currentDriverId
LEFT JOIN tb_user t2 on t2.user_id = CodOrder.signUserId
LEFT JOIN CodSendcar on CodSendcar.sendcarId = CodOrder.currentSendcarId
where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') and  CodOrder.orderTransportationType = 1
AND COdOrder.orderId not in (SELECT orderId from OnLineOrderData where indicatorsName='APP使用率')
--and CodOrder.distributionTime >= @beginTime and CodOrder.distributionTime < @endTime
--and (CONVERT(varchar(10),distributionTimeEnd,120) >= @beginTime )   and (CONVERT(varchar(10),distributionTimeEnd,120) < @endTime )  
and distributionTimeEnd >= @beginTime    and distributionTimeEnd < @endTime 
and CodOrder.currentdriverid > 0
and CodOrder.distributionTIme > CodOrder.createTime
AND  CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
AND CodClient.clientId not  in (
SELECT CodClient.clientId FROM CodClient 
LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
)
--and CodOrder.clientId not in (17953,18262,19083,19048,19327,19022,19398,19483,17992,18070,19843)
and CodOrder.deleteflag = 0   and CodOrder.orderState <> 4
order by CodOrder.distributionTime 
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细APP使用率.xlsx')

In [12]:
#天眼异常处理率汇总
sql = """
declare @beginTime date
declare @endTime date
declare @companyId int 
select @beginTime='2019-05-1', @endTime='2019-05-14'

SELECT t2.managementName as '归属',t.da as '日期',sum(orderNum)'总数', sum(t.doNum) as '已处理总数'
 FROM
(
SELECT
        tc.company_Name ,--as '公司名称',
        tc.company_id,
        CONVERT(VARCHAR(10),wmce.exceptionTime,120) as da, --as '日期',
				count(wmce.orderId) as orderNum, --'总数',
				SUM(
								CASE WHEN 
										wmce.closeState= 2 
										AND 																
											(
														convert(varchar(10),wmce.handlerTime,120) = convert(varchar(10),exceptionTime,120)
 			
											)
								THEN 1 
								ELSE 0 END
						) doNum --'已处理总数'
        FROM wj_monitor_codorder wmc 
        LEFT JOIN wj_monitor_codorder_exception wmce(nolock) ON wmc.orderId =wmce.orderId
				LEFT JOIN codorder co(nolock) on co.orderId = wmc.orderId
        LEFT JOIN CodClient cc(nolock) ON wmc.clientId =cc.clientId AND cc.deleteFlag=0
        LEFT JOIN CodSendcar cs on cs.sendcarId = co.currentSendCarId AND cs.deleteFlag=0
        left join tb_user tu on tu.user_id = co.currentDriverId AND tu.delete_flag=0
        LEFT JOIN CodDriverBindCar cbc ON tu.user_id = cbc.driver AND cbc.deleteFlag=0
        LEFT JOIN CodCarInfo cci ON cci.carInfoId = cbc.bindCarInfo AND cci.deleteFlag=0
LEFT JOIN tb_company tc ON tc.company_Id = wmce.companyId
        where
				((co.warehouseRequireTime is NULL AND  co.distributionTime > co.createTime) or co.warehouseRequireTime  > co.createTime) and
         cs.driver = co.currentDriverId
and tc.company_Id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
AND  cc.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
AND cc.clientId != (19588)
AND cc.clientId not in (select clientId from CodClient where companyId in(SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') and clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0 ) 
AND wmc.orderId not in (SELECT orderId from OnLineOrderData where indicatorsName='天眼异常')
and co.orderState != 4
--exceptionState    1:当天异常,2:即将异常
AND wmce.exceptionState in(1,2)
AND co.deleteFlag = 0
AND co.orderTransportationType != 3
-- exceptionType 1:到仓签到异常  3:签收异常
AND wmce.exceptionType in (1, 3)
AND CONVERT(VARCHAR(10),wmce.exceptionTime,120) >= @beginTime AND CONVERT(VARCHAR(10),wmce.exceptionTime,120) < @endTime

and co.deleteflag = 0 
group by CONVERT(VARCHAR(10),wmce.exceptionTime,120),tc.company_name,tc.company_id
-- order by tc.company_name,CONVERT(VARCHAR(10),wmce.exceptionTime,120)
)t
LEFT JOIN OnLineCompanyManagement t1 on t.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName ,t.da  
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/A汇总天眼异常处理率.xlsx')

In [13]:
#天眼异常处理率明细
sql = """
declare @beginTime date
declare @endTime date
declare @companyId int 
select @beginTime='2019-05-1', @endTime='2019-05-14'
SELECT
  DISTINCT
				tc.company_name '公司',
		CONVERT(VARCHAR(10),wmce.exceptionTime,120) '日期',
        wmc.orderNo AS '订单号',
        cc.clientName as '所属客户',
        tu.real_name as '司机姓名',
        cci.carLicense AS '车牌号',
        tu.user_phone AS  '手机号',
				ISNULL(co.warehouseRequireTime, co.distributionTime) as '标准到仓签到时间',
        co.warehouseSignInTime as '实际到仓签到时间',
        co.distributionTimeEnd as '标准签收时间',
        co.signTime as '实际签收时间',
        (CASE WHEN  wmce.exceptionState=1 THEN '当天异常' ELSE '即将异常' END) '异常状态',
        wmce.handlerUserName '处理人',
        wmce.handlerTime '处理时间',
				wmce.exceptionTime '异常生成时间',
        (CASE WHEN  wmce.closeState=1 THEN '未处理' ELSE '已处理' END) '处理状态',
        (CASE WHEN  wmce.exceptionType=1 THEN '到仓签到' ELSE '签收' END) '异常类型',
        wmce.handlerRemark '处理备注',
				(CASE WHEN 
										wmce.closeState= 2 
										AND 																
											(
														convert(varchar(10),wmce.handlerTime,120) = convert(varchar(10),exceptionTime,120)
											)
								THEN '正常' 
								ELSE '异常' END ) '及时处理',
		t2.managementName '归属'
        FROM
        codorder co(nolock)
        LEFT JOIN
        wj_monitor_codorder_exception wmce(nolock) ON co.orderId = wmce.orderId
        LEFT JOIN wj_monitor_codorder wmc on wmc.orderId = wmce.orderId
        LEFT JOIN CodClient cc(nolock) ON wmc.clientId =cc.clientId AND cc.deleteFlag=0
        LEFT JOIN CodSendcar cs on cs.sendcarId = co.currentSendCarId  AND cs.deleteFlag=0
        left join tb_user tu on tu.user_id = co.currentDriverId AND tu.delete_flag=0
        LEFT JOIN CodDriverBindCar cbc ON tu.user_id = cbc.driver AND cbc.deleteFlag=0
        LEFT JOIN CodCarInfo cci ON cci.carInfoId = cbc.bindCarInfo AND cci.deleteFlag=0
				LEFT JOIN tb_company tc ON tc.company_Id = wmce.companyId

				LEFT JOIN OnLineCompanyManagement t1 on tc.company_id = t1.companyId
				LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
        where
((co.warehouseRequireTime is NULL AND  co.distributionTime > co.createTime   ) 
or (co.warehouseRequireTime  > co.createTime)) AND
         cs.driver = co.currentDriverId
            and wmce.exceptionType in(1,3)
						and tc.company_Id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
						AND  cc.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')

--AND CONVERT(VARCHAR(10),wmce.exceptionTime,120) = @beginTime
AND wmc.orderId not in (SELECT orderId from OnLineOrderData where indicatorsName='天眼异常')
AND CONVERT(VARCHAR(10),wmce.exceptionTime,120) >= @beginTime AND CONVERT(VARCHAR(10),wmce.exceptionTime,120) < @endTime

						--exceptionState    1:当天异常,2:即将异常
			AND wmce.exceptionState in (1,2)
        
        and co.orderState != 4
		AND cc.clientId not in (select clientId from CodClient where companyId in(SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') and clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0 ) 
GROUP BY 
				tc.company_name,
        co.distributionTime,
        wmc.orderNo,
        cc.clientName,
        tu.real_name ,
        cci.carLicense ,
        tu.user_phone ,
         co.warehouseSignInTime ,
        co.distributionTimeEnd ,
				CONVERT(VARCHAR(10),wmce.exceptionTime,120) ,
				ISNULL(co.warehouseRequireTime, co.distributionTime) ,
				ISNULL(co.warehouseTimeEnd, co.distributionTime) ,
        co.signTime ,
        wmce.exceptionState,
        wmce.handlerUserName,
        wmce.handlerTime,
        wmce.closeState,
        wmce.exceptionType,
        wmce.handlerRemark,
				t2.managementName,
exceptionTime
order by tc.company_name,CONVERT(VARCHAR(10),wmce.exceptionTime,120);
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/A明细天眼异常处理率.xlsx')

### 及时率

In [14]:
#无单派车补录订单及时率汇总
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
	t2.managementName as '归属',
	sum(t.orderSumCount) as '无单派车补录订单数',
	sum(t.countSendCar) as '派车时间小于配送截止时间订单数',
	t.distributionTimeEnd as '派车配送截止时间'
from 
(SELECT
	cm.COMPANY_NAME,
	CodOrder.companyId as companyId,
	COUNT (CodOrder.orderId) as orderSumCount,
	sum(case when cs.sendcarTime < CodOrder.distributionTimeEnd then 1 else 0 end) as countSendCar,
	convert(varchar(10),CodOrder.distributionTimeEnd,120) as distributionTimeEnd
FROM
	CodOrder LEFT JOIN CodSendcar cs ON CodOrder.currentSendcarId=cs.sendcarId LEFT JOIN tb_company cm ON CodOrder.companyId=cm.COMPANY_ID 
WHERE
	CodOrder.deleteFlag = 0
	AND CodOrder.businessType <> '无单派车'
	and CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
	AND CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
	AND CodOrder.clientId not in (
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
	) 
	AND cs.exeTimeType = 2
	AND CodOrder.distributionTimeEnd >= @beginTime
	AND CodOrder.distributionTimeEnd < @endTime
GROUP BY CodOrder.companyId,cm.COMPANY_NAME,CodOrder.distributionTimeEnd) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.distributionTimeEnd
ORDER BY t2.managementName,t.distributionTimeEnd;
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总无单派车补录及时率.xlsx')

In [15]:
#无单派车补录订单及时率明细
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
	t2.managementName as '归属',
	t.companyName as '公司名称',
	t.clientName as '客户名称',
	t.sendCarNo as '派车单号',
	t.businessType as '派车类型',
	t.orderNo as '订单编号',
	t.createTime AS '订单创建时间',
	t.distributionTime AS '配送开始时间',
	t.distributionTimeEnd AS '配送截止时间',
	case when t.sendcarTime<t.distributionTimeEnd then '及时' else '不及时' end as '是否及时'
from
(SELECT
	cm.COMPANY_NAME as companyName,
	cm.company_id as companyId,
	cc.clientName as clientName,
	cs.sendCarNo as sendCarNo,
	CodOrder.businessType as businessType,
	CodOrder.orderNo as orderNo,
	CodOrder.createTime AS createTime,
	CodOrder.distributionTime AS distributionTime,
	CodOrder.distributionTimeEnd AS distributionTimeEnd,
	cs.sendcarTime as sendcarTime
FROM
	CodOrder LEFT JOIN CodSendcar cs ON cs.sendCarId=CodOrder.currentSendcarId LEFT JOIN tb_company cm ON CodOrder.companyId=cm.COMPANY_ID LEFT JOIN CodClient cc ON CodOrder.clientId=cc.clientId 
WHERE
	CodOrder.deleteFlag = 0
	AND CodOrder.businessType <> '无单派车'
	and CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
	AND CodOrder.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
	AND CodOrder.clientId not in (
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
	)
	AND cs.exeTimeType = 2
	AND CodOrder.distributionTimeEnd >= @beginTime
	AND CodOrder.distributionTimeEnd < @endTime) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/AAAAA明细无单派车补录及时率.xlsx')

### 完成率

In [16]:
#
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
select
t2.managementName as '归属',
a.distributionTimeEnd as '派车单虚拟订单截止时间',
sum(a.sc1) as '无单派车总数',
sum(a.sc2) as '无单派车已补录的订单的派车单总数'
 from (
SELECT
	companyId,
	cm.COMPANY_NAME,
	count(sendcar.sendcarId) AS sc1,
  (select count(1) from Codorder where deleteFlag = 1 AND businessType = '无单派车' and currentSendcarId = sendcar.sendcarId) as sc2,
	(select CONVERT (VARCHAR(10), distributionTimeEnd, 120) from Codorder LEFT JOIN CodSendcar on Codorder.currentSendcarId = CodSendcar.sendcarId where Codorder.businessType = '无单派车' and Codorder.currentSendcarId = sendcar.sendcarId AND Codorder.distributionTimeEnd > @beginTime
AND Codorder.distributionTimeEnd < @endTime) as distributionTimeEnd
FROM
	CodSendcar sendcar
LEFT JOIN tb_company cm ON sendcar.companyId = cm.COMPANY_ID
WHERE
	sendcar.deleteFlag = 0
and sendcar.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
AND sendcar.clientNos not in (select clientNo from codClient where clientId in(
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0))
and	sendcar.clientNos not in (select clientNo from CodClient where companyId in(SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') and clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0 ) 
AND sendcar.exeTimeType = 2
GROUP BY
	sendcar.companyId,
	sendcar.sendcarId,
	cm.COMPANY_NAME) a LEFT JOIN OnLineCompanyManagement t1 on a.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
where a.distributionTimeEnd>=@beginTime and a.distributionTimeEnd<=@endTime
GROUP BY t2.managementName,a.distributionTimeEnd

"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总无单完成率.xlsx')

In [17]:
#
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
  t2.managementName as '归属',
  t.clientNames '客户名称',
	t.distributionTimeEnd AS '派车单虚拟订单截止时间',
	t.sendcarNo '派车单号',
	t.isBulu as '是否补录'
from 
(SELECT
	companyId,
	cm.COMPANY_NAME as companyName,
  sendcar.clientNames as clientNames,
	sendcar.sendcarNo as sendcarNo,
	(select CONVERT (VARCHAR(10), distributionTimeEnd, 120) from Codorder LEFT JOIN CodSendcar on Codorder.currentSendcarId = CodSendcar.sendcarId where Codorder.businessType = '无单派车' and Codorder.currentSendcarId = sendcar.sendcarId AND Codorder.distributionTimeEnd > @beginTime
AND Codorder.distributionTimeEnd < @endTime) as distributionTimeEnd,
	(case when (select count(1) from Codorder where deleteFlag = 0 AND businessType = '无单派车' and currentSendcarId = sendcar.sendcarId) > 0 then '未补录' else '已补录' end ) as isBulu
FROM
	CodSendcar sendcar
LEFT JOIN tb_company cm ON sendcar.companyId = cm.COMPANY_ID
WHERE
	sendcar.deleteFlag = 0
and sendcar.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
AND sendcar.clientNos not in (select clientNo from codClient where clientId in(
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0))
and	sendcar.clientNos not in (select clientNo from CodClient where companyId in(SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') and clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0 ) 
AND sendcar.exeTimeType = 2
) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
where t.distributionTimeEnd>=@beginTime and t.distributionTimeEnd<=@endTime
GROUP BY t2.managementName,t.clientNames,t.sendcarNo,t.distributionTimeEnd,isBulu;

"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细无单派车完成率.xlsx')

### 使用率

In [18]:
#
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
SELECT 
t2.managementName as '归属',
t.companyId as '公司Id',
t.COMPANY_NAME as '公司名称',
t.createTime as '派车单创建时间',
sum(t.sc1) as '派车总数',
sum(t.sc2) as '无单派车总数'
FROM 
(select 
a.companyId as companyId,
a.COMPANY_NAME as COMPANY_NAME,
a.createTime as createTime,
sum(a.sc1) as sc1,
sum(a.sc2) as sc2
 from (
SELECT
	companyId,
	cm.COMPANY_NAME,
	CONVERT (VARCHAR(10), createTime, 120) AS createTime,
	count(sendcar.sendcarId) AS sc1,
	case when sendcar.exeTimeType=2 then 1 else 0 end as sc2
FROM
	CodSendcar sendcar
LEFT JOIN tb_company cm ON sendcar.companyId = cm.COMPANY_ID
WHERE
	sendcar.deleteFlag = 0
	AND sendcar.companyId in (select  companyId from  OnLineCompanyData where deleteFlag = 0 and indicatorsName = '无单派车补录')
	AND sendcar.clientNames in (select clientName from OnlineNoOrderClient)
	AND sendcar.createTime > @beginTime
	AND sendcar.createTime < @endTime
GROUP BY
	sendcar.companyId,
	CONVERT (VARCHAR(10), sendcar.createTime, 120),
	sendcar.sendcarId,
	cm.COMPANY_NAME,sendcar.exeTimeType) a GROUP BY a.companyId,a.COMPANY_NAME,a.createTime) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.companyId,t.COMPANY_NAME,t.createTime;
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总无单派车使用率.xlsx')

In [19]:
#
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-1', @endTime='2019-05-14'
select 
t2.managementName as '归属',
t.companyId as '公司ID',
t.companyName as '公司名称',
t.clientNames as '客户名称',
t.sendcarNo as '派车单号',
t.isNoSendCar as '是否无单派车',
t.createTime as '创建时间'
from 
(SELECT
	companyId as companyId,
	cm.COMPANY_NAME as companyName,
  sendcar.clientNames as clientNames,
	CONVERT (VARCHAR(10), createTime, 120) AS createTime,
	sendcar.sendcarNo as sendcarNo,
	(case when sendcar.exeTimeType=2 then '是' else '否' end) as isNoSendCar
FROM
	CodSendcar sendcar
LEFT JOIN tb_company cm ON sendcar.companyId = cm.COMPANY_ID
WHERE
	sendcar.deleteFlag = 0
	-- AND sendcar.companyId in (select  companyId from  OnLineCompanyData where deleteFlag = 0 and indicatorsName = '无单派车补录')
	AND sendcar.clientNames in (select clientName from OnlineNoOrderClient)
	AND sendcar.createTime > @beginTime
	AND sendcar.createTime < @endTime)t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.companyId,t.companyName,t.clientNames,t.sendcarNo,t.isNoSendCar,t.createTime;
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/AAAAAA明细无单派车使用率.xlsx')

In [20]:
#到店
sql = """
select 
t2.managementName as '归属',
t.companyId as '公司ID',
t.companyName as '公司名称',
t.clientName as '客户',
t.clientType as '客户类型',
t.daoDian as '到店是否启用',
t.miles as '到店公里限制'
from
(select 
cm.COMPANY_ID as companyId,
cm.company_name as companyName,
clientName as clientName,
(CASE when CodClient.clientTypeId=1356 THEN '配送' when CodClient.clientTypeId=1357 THEN '仓配一体' END) as clientType
,
(case when confines=2 THEN '启用'
 when confines=0 then '不启用'
ELSE '' 
end) as daoDian,
miles as miles
from CodClient LEFT JOIN tb_company cm ON CodClient.companyId=cm.COMPANY_ID
		WHERE
			deleteFlag = 0
		AND clientState = 0
		AND CodClient.businessType <> '即时业务'
		AND CodClient.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
		AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '到店')
		AND CodClient.clientId not in (
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0) 
		AND clientTypeId IN (
			SELECT
				clientTypeId
			FROM
				CodClientType
			WHERE
				
			clientTypeName IN ('配送', '仓配一体')
			AND spellName IN ('PEISONG', 'CANGPEIYITI')
			AND clientTypeNo IN ('2', '3')
			AND deleteFlag=0
))t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.companyId,t.companyName,t.clientName,t.clientType,t.daoDian,t.miles;

"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
#df.head()
df.to_excel('./data/运营/新到店电子围栏开启率.xls')

In [21]:
#到仓
sql = """
SELECT
t2.managementName as '归属',
t.companyId as '公司ID',
t.companyName as '公司名称',
t.clientName as '客户',
t.clientType as '客户类型',
t.daoCang as '到仓是否启用',
t.pickMiles as '到仓公里限制'
from
(select
cm.company_id as companyId, 
cm.company_name as companyName,
clientName as clientName,
(CASE when CodClient.clientTypeId=1356 THEN '配送' when CodClient.clientTypeId=1357 THEN '仓配一体' END) as clientType,
(CASE when CodClient.pickMconfines=1 THEN '启用'
		WHEN CodClient.pickMconfines=0 THEN '不启用'
		ELSE '' END) as daoCang,
pickMiles as pickMiles
from CodClient LEFT JOIN tb_company cm ON CodClient.companyId=cm.COMPANY_ID
		WHERE
			deleteFlag = 0
		AND clientState = 0
		AND CodClient.businessType <> '即时业务'
		AND CodClient.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
		AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '到仓')
		AND CodClient.clientId not in (
	SELECT CodClient.clientId FROM CodClient 
	LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
	WHERE 
	tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
	AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
	AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0) 
	AND clientTypeId IN (
			SELECT
				clientTypeId
			FROM
				CodClientType
			WHERE
			clientTypeName IN ('配送', '仓配一体')
			AND spellName IN ('PEISONG', 'CANGPEIYITI')
			AND clientTypeNo IN ('2', '3')
			AND deleteFlag=0
)) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.companyId,t.companyName,t.clientName,t.clientType,t.daoCang,t.pickMiles;
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
#df.head()
df.to_excel('./data/运营/新到仓电子围栏开启率.xls')

In [22]:
#明细停运客户及时关闭率
sql = """
SELECT  DISTINCT CodClient.clientId as bc,tb_company.company_id,tb_company.company_name,CodClient.clientName,CodClientType.clientTypeName  INTO  #qwer FROM
tb_company 
LEFT JOIN CodClient on CodClient.companyid = tb_company.company_id
LEFT JOIN CodClientType on CodClientType.clientTypeId = CodClient.clientTypeId AND  clientTypeName IN ('配送', '仓配一体') AND CodClientType.spellName IN ('PEISONG', 'CANGPEIYITI') --AND clientTypeNo IN ('2', '3')

where tb_company.company_Id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') 

AND  CodClient.clientId not  in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
AND CodClient.clientId not  in (
SELECT CodClient.clientId FROM CodClient 
LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
)
AND  CodClient.businessType != '即时业务'  
and CodClient.deleteflag = 0   and CodClient.clientState = 0
GROUP BY tb_company.company_id,tb_company.company_name,CodClient.clientId,CodClient.clientName,CodClientType.clientTypeName
;
SELECT   DISTINCT CodClient.clientId as ac,tb_company.company_id ,(case WHEN  COUNT(CodOrder.orderId) > 0 THEN '是' ELSE
(case WHEN CodClient.createTime > (select convert(DATE,dateadd(DD,-30,getdate()))) THEN '是' ELSE 
(case WHEN ck.ccCount > 0 THEN '是' ELSE '否' END)
 END )
END ) as state  into #asdf --FROM CodOrder 

--LEFT JOIN tb_company on tb_company.company_id = codorder.companyId
--LEFT JOIN CodClient on CodClient.clientId = CodOrder.clientId and (CodOrder.distributionTime >= (select convert(DATE,dateadd(DD,-30,getdate()))))  --and (CodOrder.distributionTime <= (select convert(DATE,dateadd(DD,-0,getdate()))))
--LEFT JOIN CodClientType on CodClientType.clientTypeId = CodClient.clientTypeId AND  clientTypeName IN ('配送', '仓配一体') AND CodClientType.spellName IN ('PEISONG', 'CANGPEIYITI') --AND clientTypeNo IN ('2', '3')
FROM     CodClient
LEFT JOIN tb_company on tb_company.company_id = CodClient.companyId
LEFT JOIN CodOrder on tb_company.company_id = CodOrder.companyId and (CodOrder.distributionTime >= (select convert(DATE,dateadd(DD,-30,getdate()))))  --and (CodOrder.distributionTime <= (select convert(DATE,dateadd(DD,-0,getdate()))))
LEFT JOIN CodClientType on CodClientType.clientTypeId = CodClient.clientTypeId AND  clientTypeName IN ('配送', '仓配一体') AND CodClientType.spellName IN ('PEISONG', 'CANGPEIYITI') --AND clientTypeNo IN ('2', '3')
LEFT JOIN (
SELECT
		SUM (sumCount) ccCount,cl.clientId
		FROM
		CcInventory ci
		LEFT JOIN CcWarehouse cw ON ci.warehouseId = cw.warehouseId
		LEFT JOIN CcGoodsAllocation cg ON (cg.goodsAllocationId = ci.goodsAllocationId AND (cg.lockingFlag IS NULL OR cg.lockingFlag = 0 ) 	)				
		LEFT JOIN CcGoodsDetail cgd ON cgd.goodsDetailId = ci.goodsDetailId
		LEFT JOIN CodClient cl ON  cgd.clientId = cl.clientId
		WHERE
		ci.companyId = 71
		AND  cl.clientState <> 1
		AND  cl.deleteFlag <> 1
		AND  cw.deleteFlag <> 1
		AND  cw.openFlag <> 1
		AND  cg.deleteFlag <> 1
		AND  cg.openFlag <> 1
GROUP BY CL.clientId
) ck on ck.clientId = CodClient.clientId
where CodOrder.companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用') -- and  CodOrder.orderTransportationType = 1
AND  CodClient.clientId not  in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
AND CodClient.clientId not  in (
SELECT CodClient.clientId FROM CodClient 
LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
WHERE 
tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0
)
-- and CodOrder.currentdriverId > 0
AND  CodClient.businessType != '即时业务'  
-- and CodOrder.distributionTime >= '2019-04-01'  and CodOrder.distributionTime <= '2019-04-08'
and CodOrder.deleteflag = 0   and CodOrder.orderState <> 4
and CodClient.deleteflag = 0   and CodClient.clientState = 0
GROUP BY tb_company.company_id,CodClient.clientId ,CodClient.createTime,ck.ccCount
;
SELECT #qwer.company_name '公司名称',#qwer.company_id '公司ID',#qwer.bc '客户ID',#qwer.clientName '客户名称', #qwer.clientTypeName '客户类型',t2.managementName '归属',
(CASE WHEN #asdf.state is null THEN '否' ELSE #asdf.state END) '是否正常发货',
( '否') '是否关闭' FROM #qwer   LEFT JOIN #asdf 
on #asdf.ac= #qwer.bc AND #qwer.company_id=#asdf.company_id

LEFT JOIN OnLineCompanyManagement t1 on #qwer.company_id = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

GROUP BY #qwer.company_id, #qwer.bc,#qwer.company_name,#qwer.clientName,#asdf.state,#qwer.clientTypeName,t2.managementName
ORDER BY #qwer.company_id
;
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/明细停运客户及时关闭率.xlsx')

### 到店地址准确

In [23]:
#到店地址准确 T-2数据，T-1,人为处理，T出数据
sql = """
薛少斌:
-- T-2地址异常上报处理率(分子：上报的未处理的数;分母：订单总数)
declare @beginTime date
declare @endTime date
select @beginTime='2019-05-1', @endTime='2019-05-13'
select 
	t2.managementName as '归属',
	t.distributionTime AS '日期',
	sum(t.addressCount) AS '未处理地址异常的订单数',
	sum(t.orderCount) AS '订单总数'
from 
(SELECT
	c3.COMPANY_NAME AS companyName,
	c3.company_id AS companyId,
	c3.distributionTime AS distributionTime,
	SUM (auditState) AS addressCount,
	COUNT (1) AS orderCount
FROM
	(
		SELECT
			c2.COMPANY_NAME,
			c2.COMPANY_ID,
			(
				CASE
				WHEN c.auditState = 0 THEN
					1
				ELSE
					0
				END
			) AS auditState,
			a.distributionTime
		FROM
			(
				SELECT
					companyId,
					orderNo,
					clientId,
					CONVERT (
						VARCHAR (10),
						distributionTime,
						120
					) as distributionTime
				FROM
					CodOrder
				WHERE
					distributionTime > @beginTime
				AND distributionTime < @endTime
				AND orderState = 3
				AND orderTransportationType = 1 
				AND companyId in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
				AND clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
				AND clientId not in (
		SELECT CodClient.clientId FROM CodClient 
		LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
		WHERE 
		tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
		AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
		AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0)
			) a
		LEFT JOIN CodAddressException c ON a.orderNo = c.orderNo and c.isComplement=0
		LEFT JOIN CodClient c1 ON a.clientId = c1.clientId
		LEFT JOIN tb_company c2 ON a.companyId = c2.company_id
        
	) c3
GROUP BY
	COMPANY_NAME,
	COMPANY_ID,
	distributionTime,
	auditState) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.distributionTime ORDER BY t2.managementName
"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df.to_excel('./data/运营/汇总到店地址异常.xlsx')

In [24]:
#到店地址准确
sql = """
declare @beginTime date
declare @endTime date
select @beginTime='2019-05-1', @endTime='2019-05-13'
SELECT
	t2.managementName as '归属',
	c2.company_name as '公司名称',
	c1.clientName as '客户名称',
	a.distributionTime as '日期',
	a.orderNo as '订单编号',
	a.endLinkman as '收货人',
	a.endLinkmanNo as '收货人编码',
	t.real_Name as '司机姓名',
	t.user_phone as '司机电话',
	(case when c.addressExceptionId is null then '否' else '是' end) as '是否上报',
	(case when c.auditState = 0 then '未审核' when c.auditState = 1 then '审核通过' when c.auditState = 2 then '审核驳回' end) as '审核状态',
	(case when c.correctionState = 0 then '未校正' when c.correctionState = 1 then '已校正' end) as '是否校正'
FROM
	(
		SELECT
			clientId,
			companyId,
			orderNo,
			endLinkman,
			endLinkmanNo,
			currentDriverId,
			CONVERT (
				VARCHAR (10),
				distributionTime,
				120
			) as distributionTime
		FROM
			CodOrder
		WHERE
			distributionTime > @beginTime
		AND distributionTime < @endTime
		AND orderState = 3
		AND orderTransportationType = 1 
		AND companyId IN (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
        and clientId not in(SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
				AND clientId not in (
SELECT CodClient.clientId FROM CodClient 
		LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId
		WHERE 
		tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部')
		AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部')
		AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0)) a
LEFT JOIN tb_user t ON t.user_Id = a.currentDriverId
LEFT JOIN CodAddressException c ON a.orderNo = c.orderNo and c.isComplement=0
LEFT JOIN CodClient c1 ON a.clientId=c1.clientId 
LEFT JOIN tb_company c2 ON a.companyId=c2.company_id 
LEFT JOIN OnLineCompanyManagement t1 on a.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId;

"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df
df.to_excel('./data/运营/明细到店地址异常.xlsx')

### 到仓地址准确

In [27]:
#到仓地址准确
sql = """
-- 到仓异常(后台处理的上报异常的订单数量/订单数量)
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-01', @endTime='2019-05-14'
select 
	t2.managementName as '归属',
	t.distributionTime as '配送时间',
	sum(orderCount) as '订单数量',
	sum(t.exceptionCount) as '到仓上报异常的数量'
from 
(select 
	  CodOrder.companyId as companyId,
		CONVERT (
						VARCHAR (10),
						distributionTime,
						120
					) as distributionTime,
	  1 as orderCount,
	  (CASE when (select DISTINCT(orderNo) as orderNo from CodOrderFenceException where CodOrderFenceException.orderNo=CodOrder.orderNo and CodOrderFenceException.orderNo in(select orderNo from CodOrder where deleteFlag=0 and pickUpUserId<>currentDriverId and CodOrder.distributionTime>@beginTime and CodOrder.distributionTime<@endTime)) is null then 0 ELSE 1 end) as exceptionCount
from CodOrder where deleteFlag=0 
		and orderId not in (select orderId from OnLineOrderData where deleteFlag=1 and indicatorsName='仓库地址异常')
		and companyId IN (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
        and orderTransportationType = 1 
		and clientId not in(SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
		AND clientId not in (SELECT CodClient.clientId FROM CodClient LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId WHERE tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0)
		and CodOrder.distributionTime>@beginTime and CodOrder.distributionTime<@endTime and (CodOrder.orderState=2 or CodOrder.orderState=3)) t LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId
GROUP BY t2.managementName,t.distributionTime;


"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df
df.to_excel('./data/运营/汇总到仓地址异常.xlsx')

In [28]:
#到仓地址准确
sql = """
declare @beginTime varchar(20)
declare @endTime varchar(20)
select @beginTime='2019-05-01', @endTime='2019-05-14'
select 
	t2.managementName as '归属',
    t.distributionTime as '配送时间',
	cm.company_name as '公司名称',
	cc.clientName as '客户名称',
	t.orderNo as '订单编号',
(CASE when (select DISTINCT(orderNo) as orderNo from CodOrderFenceException where CodOrderFenceException.orderNo=t.orderNo and CodOrderFenceException.orderNo in(select orderNo from CodOrder where deleteFlag=0 and pickUpUserId<>currentDriverId and CodOrder.distributionTime>@beginTime and CodOrder.distributionTime<@endTime)) is null then '否' ELSE '是' end) as '到仓地址是否异常'
from 
(select 
	  CodOrder.companyId as companyId,
		CodOrder.clientId as clientId,
		CodOrder.orderNo as orderNo,
        CONVERT (
						VARCHAR (10),
						CodOrder.distributionTime,
						120
					) as distributionTime
from CodOrder where  deleteFlag=0 
		and orderId not in (select orderId from OnLineOrderData where deleteFlag=1 and indicatorsName='仓库地址异常')
		and companyId IN (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '通用')
        and orderTransportationType = 1 
		and clientId not in(SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '通用')
		AND clientId not in (SELECT CodClient.clientId FROM CodClient LEFT JOIN tb_company ON tb_company.company_id = CodClient.companyId WHERE tb_company.company_id in (SELECT companyId from OnLineCompanyData where DELETEFLAG = 0 AND indicatorsName = '配送本部') AND CodClient.clientId not in (SELECT clientId from OnLineClientData where DELETEFLAG = 1 AND indicatorsName = '配送本部') AND CodClient.deleteFlag = 0 AND CodClient.clientState = 0)
		and CodOrder.distributionTime>@beginTime and CodOrder.distributionTime<@endTime and (CodOrder.orderState=2 or CodOrder.orderState=3)) t LEFT JOIN tb_company cm ON t.companyId=cm.COMPANY_ID LEFT JOIN CodClient cc ON t.clientId=cc.clientId LEFT JOIN OnLineCompanyManagement t1 on t.companyId = t1.companyId
LEFT JOIN OnLineManagement t2 on t1.managementId = t2.managementId

"""
df0 = pd.read_sql(sql,conn)
df = pd.DataFrame(df0)
df
df.to_excel('./data/运营/明细到仓地址异常.xlsx')